In [ ]:
!wget https://food-x.s3.amazonaws.com/train.tar

In [ ]:
!tar -xvf train.tar

In [ ]:
%cd train_set

/content/train_set


In [ ]:
from PIL import Image
import glob
images = []
for filename in glob.glob('/content/train_set/*.jpg'): 
    im=Image.open(filename)
    images.append(im)

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.autograd import Variable
from PIL import Image

In [ ]:
def preprocess(img):
  scaler = transforms.Resize((224,224))
  normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                   std=[0.229, 0.224, 0.225])
  to_tensor = transforms.ToTensor()
  t_img = Variable(normalize(to_tensor(scaler(img))).unsqueeze(0))
  return t_img

In [ ]:
preprocess_image = []
for img in images:
  t_img = preprocess(img)
  preprocess_image.append(t_img)

In [ ]:
image_tensor = torch.Tensor(len(preprocess_image), 3, 224, 224)
torch.cat(preprocess_image, out=image_tensor)

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
from torch.autograd import Variable
resnet50 = models.resnet50(pretrained=True)
modules=list(resnet50.children())[:-1]
resnet50=nn.Sequential(*modules)
#for p in resnet50.parameters():
   # p.requires_grad = True

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth


In [ ]:
resnet50(image_tensor).size()

torch.Size([10, 2048, 1, 1])

In [ ]:
image_tensor.size()

torch.Size([10, 3, 224, 224])

In [ ]:
training_data = []
for image in image_tensor:
  training_data.append((torch.rand(2048),image))


In [ ]:
recipes = torch.rand(10,2048)

In [ ]:
from torch.nn.utils.clip_grad import clip_grad_norm_
from torch.optim import Adam, SGD
lr = 0.1
num_epochs = 20

In [ ]:
model = resnet50
loss_function = nn.CosineEmbeddingLoss()
params = list(model.parameters())
optimizer = Adam(params, lr=lr, weight_decay=5e-4)
for epoch in range(num_epochs):  
    print('Epoch {}/{}'.format(epoch, num_epochs - 1))
    for recipe, image in training_data[:1]:
        model.zero_grad()
        image = image.unsqueeze(0)
        output = model(image)
        output = torch.squeeze(output)
        loss = loss_function(output.unsqueeze(0),recipe.unsqueeze(0), torch.tensor([1]))
        loss.backward()
        optimizer.step()
    print(loss)

In [ ]:
y = torch.empty(10).bernoulli_().mul_(2).sub_(1)
y

tensor([ 1., -1., -1.,  1., -1., -1.,  1.,  1.,  1., -1.])

In [ ]:
model = resnet50
loss_function = nn.CosineEmbeddingLoss()
params = list(model.parameters())
optimizer = Adam(params, lr=lr, weight_decay=5e-4)
for epoch in range(num_epochs):  
    print('Epoch {}/{}'.format(epoch, num_epochs - 1))
    #for recipe, image in training_data[:1]:
    model.zero_grad()
    #image = image.unsqueeze(0)
    output = model(image_tensor)
    output = torch.squeeze(output)
    loss = loss_function(output,recipes, y)
    loss.backward()
    optimizer.step()
    print(loss)

Epoch 0/19
tensor(0.4075, grad_fn=<MeanBackward0>)
Epoch 1/19
tensor(0.4106, grad_fn=<MeanBackward0>)
Epoch 2/19
tensor(0.4116, grad_fn=<MeanBackward0>)
Epoch 3/19
tensor(0.4026, grad_fn=<MeanBackward0>)
Epoch 4/19
tensor(0.3876, grad_fn=<MeanBackward0>)
Epoch 5/19
tensor(0.3679, grad_fn=<MeanBackward0>)
Epoch 6/19
tensor(0.3376, grad_fn=<MeanBackward0>)
Epoch 7/19
tensor(0.3864, grad_fn=<MeanBackward0>)
Epoch 8/19
tensor(0.4330, grad_fn=<MeanBackward0>)
Epoch 9/19
tensor(0.4001, grad_fn=<MeanBackward0>)
Epoch 10/19
tensor(0.4621, grad_fn=<MeanBackward0>)
Epoch 11/19
tensor(0.3849, grad_fn=<MeanBackward0>)
Epoch 12/19
tensor(0.3893, grad_fn=<MeanBackward0>)
Epoch 13/19
tensor(0.3665, grad_fn=<MeanBackward0>)
Epoch 14/19
tensor(0.4013, grad_fn=<MeanBackward0>)
Epoch 15/19
tensor(0.4296, grad_fn=<MeanBackward0>)
Epoch 16/19
tensor(0.4139, grad_fn=<MeanBackward0>)
Epoch 17/19
tensor(0.3925, grad_fn=<MeanBackward0>)
Epoch 18/19
tensor(0.3761, grad_fn=<MeanBackward0>)
Epoch 19/19
tensor(0.3